# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* portfolio.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [92]:
import pandas as pd
import numpy as np
import ast
import re
import math
import json
%matplotlib inline


In [ ]:
# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

# Assess data

## Assess and clean PORTFOLIO

In [63]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [ ]:
def split_strlist_into_cols(df, col):
    ''' 
    For an entire Pandas dataframe split up list of values into one-hot-encoded separated columns
    with common column name prefix <col>_
    
    INPUT
    df - Pandas dataframe 
    col - Column name within dataframe
    
    OUTPUT
    df_result - Dataframe with multivalue column normalized into separate columns
    '''   
    # One-hot encode the col column
    encoded_df = df[col].apply(lambda x: pd.Series(1, index=x)).fillna(0).astype(int)
    # Add prefix to the new columns
    encoded_df = encoded_df.add_prefix(col+'_')
    
    # Replace old column by new one-hot-encoded columns
    df_result = df.drop(col, axis=1).join(encoded_df)
    return df_result


def one_hot_encode(df, col):
    ''' 
    For an entire Pandas dataframe one-hot-encode a column with singular values
    into columns with common column name prefix <col>_
    
    INPUT
    df - Pandas dataframe 
    col - Column name within dataframe
    
    OUTPUT
    df_result - Dataframe with nominal value column normalized into separate columns (one-hot-encoded)
    '''   
    # One-hot encode the col column
    encoded_df = pd.get_dummies(df[col], prefix=col+'_').astype(int)
    
    # Replace old column by new one-hot-encoded columns
    df_result = df.drop(col, axis=1).join(encoded_df)
    return df_result

portfolio_one_hot = split_strlist_into_cols(portfolio, 'channels')
portfolio_one_hot = one_hot_encode(portfolio_one_hot, 'offer_type')

# Convert duration in days into hourly values.
portfolio_one_hot['duration_hours'] = portfolio_one_hot['duration'] * 24

portfolio_one_hot



,reward,difficulty,duration,id,channels_email,channels_mobile,channels_social,channels_web,offer_type__bogo,offer_type__discount,offer_type__informational,duration_hours
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0,1,0,0,168
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,1,0,0,120
2,0,0,4,3f207df678b143eea3cee63160fa8bed,1,1,0,1,0,0,1,96
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1,1,0,0,168
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1,0,1,0,240
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1,0,1,0,168
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1,0,1,0,240
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,1,1,1,0,0,0,1,72
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1,1,0,0,120
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,1,1,0,1,0,1,0,168


## Assess relationship data quality PROFILE - TRANSCRIPT

In [20]:
portfolio.to_excel('data/portfolio.xlsx')
profile.to_excel('data/profile.xlsx')
transcript.to_excel('data/transcript.xlsx')

In [9]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


In [10]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [83]:
print(f"transcript - number of records: {len(transcript)}")

transcript - number of records: 306534


### Relationship Profile - Transcript

In [84]:
# Perform the inner join
merged_df = pd.merge(profile, transcript, left_on='id', right_on='person')

# Select the required columns
result_df = merged_df[['id', 'gender', 'age', 'became_member_on', 'income', 'event', 'value', 'time']]

result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                306534 non-null  object 
 1   gender            272762 non-null  object 
 2   age               306534 non-null  int64  
 3   became_member_on  306534 non-null  int64  
 4   income            272762 non-null  float64
 5   event             306534 non-null  object 
 6   value             306534 non-null  object 
 7   time              306534 non-null  int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 18.7+ MB


In [85]:
print(f"transcript inner join profile - number of records: {len(result_df)}")

transcript inner join profile - number of records: 306534


*As number of records of transcript is equal to number of records transcript inner join profile (306534) there are no records in transcript having no person in profile.*

### Relationship Portfolio - Transcript

In [93]:
def split_dict_into_cols(df, col):
    ''' 
    For an entire Pandas dataframe normalize the dictionary value pairs from
    column col into individual columns named by key value.
    
    INPUT
    df - Pandas dataframe 
    col - Column name within dataframe
    
    OUTPUT
    df_result - Dataframe with multivalue column normalized into separate columns
    '''   
    
    # Function to correct malformed dictionary strings
    def correct_dict_string(s):
        s = re.sub(r'(\w+):', r'"\1":', s)  # Add quotes around keys
        return s

    # Correct the dictionary strings in the specified column
    df[col] = df[col].apply(correct_dict_string)
    
    
    # Convert the string representation of dictionaries in the 'value' column to actual dictionaries
    df[col] = df[col].apply(ast.literal_eval)

    # Normalize the dictionary values in the 'value' column and join them back to the original dataframe
    df_expanded = df.join(pd.json_normalize(df[col]))

    # Drop the original 'value' column as it's no longer needed
    # df_expanded = df_expanded.drop(columns=[col])
    return df_expanded

transcript_norm = split_dict_into_cols(transcript, 'value')

transcript_norm.tail(20)



TypeError: expected string or bytes-like object, got 'dict'

In [86]:
transcript.tail(20)

,person,event,value,time
306514,f1e4fd36e5a0446f83861308bddf6945,transaction,{'amount': 8.2},714
306515,0b64be3b241c4407a5c9a71781173829,transaction,{'amount': 2.6},714
306516,86d03d35d7e0434b935e7743e83be3a0,transaction,{'amount': 9.2},714
306517,3408fd05c781401f8442fb6dbaaea9c7,transaction,{'amount': 11.7},714
306518,1593d617fac246ef8e50dbb0ffd77f5f,transaction,{'amount': 40.67},714
306519,f1b31d07b5d84f69a2d5f1d07843989e,transaction,{'amount': 31.13},714
306520,2ce987015ec0404a97ba333e8e814090,transaction,{'amount': 1.6400000000000001},714
306521,2e33545f0a764d27b2ccff95fc8d72c4,transaction,{'amount': 17.35},714
306522,d1c4500ace2e45e9a45d3cd2fccac8d8,transaction,{'amount': 4.42},714
306523,b65affd9e07346a1906364a396950e3d,transaction,{'amount': 18.35},714
